# CAD(2変数バージョン)

このスクリプトでは, CADを次の3つの段階に分けて行う.

- 射影段階: CADを与える多項式族の次数を下げる.
- 底段階:
- 持ち上げ段階:

まずは簡単のために多項式の**変数が2個の場合に限り**実装する.
その場合, 射影段階と持ち上げ段階の操作は1回で済む.

In [1]:
x,y = var("x y")
R = PolynomialRing(PolynomialRing(QQ,"y"),"x") # R=R[y][x]のこと.

# ここにCADしたい多項式のリストを渡す
#F = [x^2+y^2-1,x^2-y^3] 
F = [x^2+y^2-1,x^2-y^3]

## 射影段階

2変数多項式の族$ F $が与えられたとき, 1変数多項式の族$ F_1 \cup F_2 \cup F_3 $を次のように構成する.

- $ F_1 $: $ f \, (f \in F) $の変数yに対する係数の多項式全体
- $ F_2 $: $ \mathrm{gcd}(f, df/dx) \, (f \in F) $の変数yに対する係数の多項式全体
- $ F_3 $: $ \mathrm{gcd}(f, g) \, (f \neq g \in F) $の変数yに対する係数の多項式全体

In [2]:
# 変数xについてk次以下の部分を取り出す関数
def T(k,f):
    return f.mod(R(x^(k+1)))

# 与えられた関数をmonicにする

In [3]:
def projection(F:list):
    F = [R(f) for f in F]
    G = []
    
    # step1: F_1の計算
    for f in F:
        for term in f.coefficients():
            term = term/term.lc() # 先頭項を1にする(こういうのは2変数でしかダメっぽい)
            if term.degree()>0 and term not in G:
                G.append(term)
    # step2: F_2の計算 
    for f in F:
        for l in range(f.degree()+1):
            for k in range(l, f.degree()+1):
                for i in range(l+1):
                    if T(k,f).diff()==0:
                        continue
                    pscs = [term.leading_coefficient() for term in T(k,f).subresultants(T(k,f).diff())]
                    pscs = [term/term.lc() for term in pscs] # 先頭項を1にする
                    for psc in pscs: 
                        if psc.degree()>0 and psc not in G:
                            G.append(psc)
    # step3: F_3の計算
    for f in F:
        for g in F:
            if f==g:
                continue
            for l in range(min(f.degree(),g.degree())+1):
                for k in range(l,f.degree()+1):
                    for j in range(l, f.degree()+1):
                        if T(k,f)==0 or T(j,g)==0:
                            continue
                        pscs = [term.leading_coefficient() for term in T(k,f).subresultants(T(j,g))]
                        pscs = [term/term.lc() for term in pscs] # 先頭項を1にする
                        for psc in pscs:
                            if psc.degree()>0 and psc not in G:
                                G.append(psc)
    
    return G

In [4]:
#実験
G = projection(F);G

[y^2 - 1, y^3, y^6 + 2*y^5 + y^4 - 2*y^3 - 2*y^2 + 1, y^3 + y^2 - 1]

## 底段階

射影段階で求めた1変数多項式の族 $ G $ 不変な $ \mathbb{R} $ のセル分割を与える.
- 根の絶対値の限界を係数から評価する.
- "実根の分離"をすべて求める.
- 実根を解にもつ既約多項式を求める.


まず, スツルム列を用いた, 多項式 $ f(x) $ の $ (a,b] $ 上の実根の数を返す関数`count_root`を定義する.<p>
$ f(a) \neq 0, f(b) \neq 0 $ を満たしている必要があるので注意が必要である.

In [5]:
# 多項式f(x)の(a,b]上の異なる実根の数を返す関数
def count_root(f, a, b):
    if f in RR:
        return 0
    
    # スツルム列
    S = [f, f.diff()]
    
    while(True):
        if S[-2] % S[-1] == 0:
            break
        else:
            S.append(- (S[-2] % S[-1]))

    S_a = [s(a) for s in S]
    S_b = [s(b) for s in S]
    
    return v(S_a) - v(S_b)

# 実数列lstから符号の変化数v(lst)を計算する        
def v(lst):
    lst_sign = [bool(x>0) for x in lst if x!=0] #lstから符号列を計算
    count = 0
    for i in range(1,len(lst_sign)):
        if lst_sign[i] != lst_sign[i-1]:
            count += 1
    return count

多項式のからその解の絶対値の上界を計算する`bound_sol(f)`を定義する.

In [6]:
def bound_sol(f):
    f = f/f.lc()
    lst = f.coefficients(sparse=False)
    return 1 + max(abs(a/lst[-1]) for a in lst[:-1])

底段階の実装

In [7]:
def base(F:list, eps = 10^(-5)):
    # Fを既約多項式に分解して新たにIEE_Fとする:
    F = [f(y) for f in F]
    IRR_F = []
    for f in F:
        IRR_F += [g[0] for g in f.factor_list() if g[0] not in RR ]
    IRR_F = [PolynomialRing(QQ,"y")(f)for f in set(IRR_F)]
    
    # 解の絶対値の上界Mを求める
    M=max(bound_sol(f) for f in IRR_F)
    
    # 各既約多項式の解を求める.
    ans = []
    lst = [(-M,M,IRR_F)]
    while(len(lst)!=0):
        a,b,F = lst.pop()
        if(b-a<eps and len(F)==1):
            ans.append((a,b,F))
            continue
        if(len(F)==0):
            continue
        else:
            c = (a+b)/2
            F1 = [f for f in F if count_root(f,a,c)>0]
            F2 = [f for f in F if count_root(f,c,b)>0]
            if len(F1)>0:
                lst.append((a,c,F1))
            if len(F2)>0:
                lst.append((c,b,F2))
    ans = sorted([(a,b,F[0]) for a, b, F in ans])
    return [(f,a,b) for a, b, f in ans]
            

In [8]:
# 実験
Sol = base(G);Sol

[(y + 1, -131073/131072, -1),
 (y, -1/131072, 0),
 (y^3 + y^2 - 1, 98943/131072, 773/1024),
 (y - 1, 131071/131072, 1)]

## 持ち上げ段階(未実装)

底段階で求めたセル分割を, $ \mathbb{R}^2 $ のセル分割に持ち上げる.

- 底段階で取得した解から, sample points を取得する.
- サンプルxが有理数の場合→まったく同様にしてyのsamplepointを取得する.
- サンプルxが無理数の場合→ここは改良が必要. だが, やることは同じ.

In [9]:
def first_decomp(Sol:list, eps = 10^(-5)):
    # Solutionから, R^1のセル分割を行う.
    R = PolynomialRing(QQ, "y")
    ans = []
    if len(Sol)==0:
        ans += [(R(y), 0, 0)]
        
    elif len(Sol)==1:
        f,a,b = Sol[0]
        ans += [(R(y-(a-1)), a-1, a-1), (f,a,b), (R(y-(b+1)), b+1, b+1)]
        
    else:
        f_prev, a_prev, b_prev = Sol[0]
        ans += [(R(y-(a_prev-1)), a_prev-1, a_prev-1)]
        ans += [(f_prev, a_prev, b_prev)]
        for f_new, a_new, b_new in Sol[1:]:
            c = (b_prev + a_new)/2
            ans += [(PolynomialRing(QQ, "y")(y-c),c,c)]
            ans += [(f_new,a_new,b_new)]
            f_prev , a_prev, b_prev = f_new, a_new, b_new
        ans += [(R(y-(b_new+1)), b_new+1, b_new+1)]
    
    return ans    

In [11]:
# 実験
Cells = first_decomp(Sol); Cells

[(y + 262145/131072, -262145/131072, -262145/131072),
 (y + 1, -131073/131072, -1),
 (y + 131073/262144, -131073/262144, -131073/262144),
 (y, -1/131072, 0),
 (y - 98943/262144, 98943/262144, 98943/262144),
 (y^3 + y^2 - 1, 98943/131072, 773/1024),
 (y - 230015/262144, 230015/262144, 230015/262144),
 (y - 1, 131071/131072, 1),
 (y - 2, 2, 2)]

ここまででR^1のセル分割はできたはず. なので, 次は各サンプルポイントにおけるFの解を取得する.
- 有理数, つまり, 既約多項式の次数が1の場合は全く同様にしてセル分割ができるはず.
- 有理数でない場合, 符号判定の際にちょっとだけ特殊な方法を使わなければならない. 

```python
    for sample_point in cell:
        if sample_point in QQ:
            F_sub = [f(y=sample_point) for f in F]
            first_decomp(base(F_sub)) # これでできたリストを新しく追加する.
        if sample_point not in QQ:
            F_sub = [f(y=sample_point) for f in F] # これは有理数係数多項式のリスト. 
```
> これ以降はどうすればよいかの考察:
> - 代入した時点でQ(alpha)係数の多項式リスト$Fsub \subset \mathbb{Q}(\alpha)[x]$が与えられる.
> - 各多項式を既約多項式に分解する。係数が$\mathbb{Q}(\alpha)$なので, 実際にどうすればよいかわからないが.
> - **具体的にわからないこと:**
>   + **既約多項式に分解するアルゴリズム**
>   + **Q(alpha)上既約な多項式g(x)から, Q上既約な多項式h(x)に変換する方法**
> - 各既約多項式に対して, それらの解がすべて含まれるような上界Mを取得する.
> - 既約多項式の集合に対して, 二分探索的に解を取得していく.

In [13]:
def lifting(cells:list, F:list):
    ans = [] # ここにセルのリストを設定する.
    R = PolynomialRing(QQ, "x")
    for f,a,b in cells:
        if f.degree()==1:
            y_0 = -f.constant_coefficient()/f.lc()
            F_lift = [R(g(y=y_0)) for g in F]
            new_cells = [(R(g(y=x)),c,d) for g,c,d in first_decomp(base(F_lift))]
            ans += [((g,c,d),(f,a,b)) for (g,c,d) in new_cells]
        else:
            pass
            ################################# ここを書く #################################
            
    return ans

In [15]:
def show(cells:list):
    ans=[]
    for cell in cells:
        (f,a,b),(g,c,d) = cell
        if f.degree()==1:
            x_0=-f.constant_coefficient()/f.lc()
        else:
            x_0 = b
        if g.degree()==1:
            y_0 = -g.constant_coefficient()/f.lc()
        else:
            y_0 = d
        ans += [(x_0,y_0)]
    print(ans)


In [22]:
show(lifting(Cells, F))

[(0, -262145/131072), (-131073/131072, -1), (0, -1), (1, -1), (-525239609194517/281474976710656, -131073/262144), (-15600816220144961/18014398509481984, -131073/262144), (0, -131073/262144), (243764632483861/281474976710656, -131073/262144), (525239609194517/281474976710656, -131073/262144), (-262145/131072, 0), (-1, 0), (-131073/262144, 0), (0, 0), (131071/262144, 0), (1, 0), (2, 0), (-17348183297260481/9007199254740992, 98943/262144), (-16681840435802755/18014398509481984, 98943/262144), (-10429580845600215/18014398509481984, 98943/262144), (-1044298401540363/4503599627370496, 98943/262144), (0, 98943/262144), (4177321255397675/18014398509481984, 98943/262144), (10429580845600215/18014398509481984, 98943/262144), (8340984042519489/9007199254740992, 98943/262144), (17348183297260481/9007199254740992, 98943/262144), (-8603728829442129762865/4722366482869645213696, 230015/262144), (-1940666081396659906905/2361183241434822606848, 230015/262144), (-768328098638338563345/118059162071741130

46